In [1]:
%pip install multion
%pip install agentops
%pip install python-dotenv

  Using cached multion-1.1.0-py3-none-any.whl.metadata (4.5 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
Using cached multion-1.1.0-py3-none-any.whl (39 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import multion
from multion.client import MultiOn
import agentops
from agentops import ActionEvent, LLMEvent, ToolEvent, ErrorEvent
import os
import dotenv
from dotenv import load_dotenv

load_dotenv()

agentops.init(api_key=os.environ.get("AGENTOPS_API_KEY"))
multion = MultiOn(api_key=os.environ.get("MULTION_API_KEY"))
create_session_params= {
    "url": "https://www.rei.com/"
}
action_event = ActionEvent(params=create_session_params,action_type="create_session")
create_session_response = multion.sessions.create(**create_session_params)
action_event.returns = create_session_response.__dict__
agentops.record(action_event)
session_id = create_session_response.session_id
print(create_session_response.message)
cmd="find the top reviewed satellite phone on REI and add it to my cart. Dismiss any modals as necessary."
step_session_params= {
   	"session_id": session_id,
    "cmd": cmd,
    "url": "https://www.rei.com/",
    "include_screenshot":True,
}
llm_event = LLMEvent(prompt=cmd)
agentops.record(llm_event)

action_event = ActionEvent(params=step_session_params,action_type="step_session")
llm_event = LLMEvent()
step_session_response = multion.sessions.step(**step_session_params)
print(step_session_response.message)
action_event.returns = step_session_response.__dict__
action_event.screenshot = step_session_response.screenshot
llm_event.prompt = step_session_response.message
agentops.record(action_event)
agentops.record(llm_event)

while step_session_response.status == 'CONTINUE':
  action_event = ActionEvent(params=step_session_params,action_type="step_session")
  llm_event = LLMEvent()
  step_session_response = multion.sessions.step(**step_session_params)
  print(step_session_response.message)
  action_event.returns = step_session_response.__dict__
  action_event.screenshot = step_session_response.screenshot
  llm_event.prompt = step_session_response.message
  agentops.record(action_event)
  agentops.record(llm_event)
  
if step_session_response.status == 'DONE':
    action_event = ActionEvent(params=step_session_params,action_type="step_session")
    llm_event = LLMEvent()
    print('task completed')
    action_event.returns = step_session_response.__dict__
    action_event.screenshot = step_session_response.screenshot
    llm_event.prompt = step_session_response.message
    agentops.record(action_event)
    agentops.record(llm_event)

print(step_session_response.message)
get_screenshot = multion.sessions.screenshot(session_id=session_id)
print("screenshot of session: ", get_screenshot.screenshot)
close_session_response = multion.sessions.close(session_id=session_id)
print("close_session_response: ", close_session_response)

agentops.end_session("Success")

ImportError: cannot import name 'load_env' from 'dotenv' (/Users/howardgil/Desktop/agentops/AgentOps-AI/agentops/env/lib/python3.12/site-packages/dotenv/__init__.py)